## Laden standard Packages

In [ ]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
import matplotlib.pyplot as plt
import matplotlib.dates as dates
%matplotlib inline
import math
from dateutil.relativedelta import relativedelta

## Datenaufbereitung


In [ ]:
# Laden der Daten
import os

data_path='/domino/datasets/local/TestData/iboxx_eur/'

## Komponenten als monatsweises Dictionary von DataFrames 

comp_list=os.listdir(data_path+'Components/')

comp={}

for file in comp_list:
    comp[file[-10:-4]]=pd.read_csv(data_path+'Components/'+file,encoding='latin_1')


## Indizes als ein GesamtDataframe auf täglicher Basis

indices_list=os.listdir(data_path+'Indices')

indices={}

for file in indices_list:
    indices[file[-12:-4]]=pd.read_csv(data_path+'Indices/'+file,encoding='latin_1')

i=0

for days in indices:
    if i==0:
        indices_df=pd.DataFrame(indices[days])
    else:
        indices_df=indices_df.append(indices[days],ignore_index=True)
    i=i+1

## Underlyings als ein GesamtDataframe auf täglicher Basis

underlyings_list=os.listdir(data_path+'Underlyings')

underlyings={}

for file in underlyings_list:
    underlyings[file[-12:-4]]=pd.read_csv(data_path+'Underlyings/'+file,encoding='latin_1')

## Underlyings als ein GesamtDataframe auf täglicher Basis

underlyings_list=os.listdir(data_path+'Underlyings')

underlyings={}

for file in underlyings_list:
    underlyings[file[-12:-4]]=pd.read_csv(data_path+'Underlyings/'+file,encoding='latin_1')
    
underlyings_df=pd.DataFrame()

for days in underlyings:
    underlyings_df=underlyings_df.append(underlyings[days][['Date','ISIN','Index Price','Daily Return']],ignore_index=True)

## Abtrennung Trainingsdatensatz

In [ ]:
Startdatum='2019-12-01'
Enddatum='2021-03-31'

IndexISIN='DE0009682716'

N=50

IndexDaten=indices_df[(indices_df['ISIN_TRi']==IndexISIN) & 
                      (indices_df['Date']>=Startdatum) & 
                      (indices_df['Date']<=Enddatum)].sort_values('Date',ignore_index=True)
UnderlyingDaten=underlyings_df[(underlyings_df['Date']>=Startdatum) & 
                      (underlyings_df['Date']<=Enddatum)].sort_values(['Date','ISIN'],ignore_index=True)


## Funktion zur Bestimmung der Startkomponenten

In [ ]:
def StartComp(N=30,Components=comp,Year=2020,Month=11):
    key=str(Year)
    
    if len(str(Month))==2:
        key=key+str(Month)
    else:
        key=key+'0'+str(Month)
        
        ISIN_Set=set(Components[key].sort_values('Index Weight').iloc[0:N]['ISIN'])
        
        Weights=np.array(Components[key]['Index Weight'])
        
        w=np.where(np.isin(Weights,np.array(Components[key].sort_values('Index Weight').iloc[0:N]['Index Weight'])),Weights,0)
        w=w/w.sum()
    
    return (ISIN_Set,w)

## Optimierung über non negative least squares (NNLS)

Optimierungsproblem: Minimiere Tracking Error unter den Nebenbedingungen, dass die Gewichte größer als 0 sein müssen und nur ein Teil der Komponenten verwendet werden sollen um Kosten zu sparen.
Aufbau: Anhand der Daten der letzten Monate werden die Gewichte für den kommenden Monat geschätzt.



In [ ]:
from scipy.optimize import nnls

#### Bestimme Monat in n Monaten

In [ ]:
def MonthDelta(Year,Month,n):
    
    Month_new=Month+n
    
    if Month_new>12 or Month_new<1:
        Month=Month_new%12
        Year=Year+Month_new//12
        if Month==0:
            Month=12
            Year=Year-1
    else:
        Month=Month_new
    
    return (Year,Month)

### Einfache optimierung über Preis

Funktionen erlauben ein Optimierung über eine Periode und deren iterative Verwendung

In [ ]:
def SinglePeriodPrice(N,Components,Index,Underlying,Year,Month,NMonth=6,i=0):
    ActComp=StartComp(N,Components,Year,Month)
    #print(Year,'-',Month,':',ActComp)
    
    IndexPrice=Index[['Date','Gross Price Index']].set_index('Date')
    UnderlyingPrice=Underlying[Underlying['ISIN'] \
                               .isin(ActComp)][['Date','ISIN','Index Price']] \
                               .pivot(index='Date',columns='ISIN',values='Index Price')
    
    Prices=pd.merge(IndexPrice,UnderlyingPrice,left_index=True,right_index=True)
    Prices=Prices.fillna(method='ffill',axis=0).fillna(method='bfill',axis=0)
    (YearTrain,MonthTrain)=MonthDelta(Year,Month,-NMonth)
    Prices_train=Prices[((pd.to_datetime(Prices.index).year<Year)| \
                  ((pd.to_datetime(Prices.index).month<=Month) & (pd.to_datetime(Prices.index).year==Year))) & \
                       ((pd.to_datetime(Prices.index).year>YearTrain)| \
                  ((pd.to_datetime(Prices.index).month>=MonthTrain) & (pd.to_datetime(Prices.index).year==YearTrain)))]
    (YearTest,MonthTest)=MonthDelta(Year,Month,1)
    Prices_test=Prices[(~Prices.index.isin(Prices_train.index)) & \
                       (pd.to_datetime(Prices.index).month==MonthTest) & \
                       (pd.to_datetime(Prices.index).year==YearTest)]
    
    
    b=np.array(Prices_train.iloc[:,0])
    A=np.array(Prices_train.iloc[:,1:N+1])
    
    (x, rnorm)=nnls(A,b)
    
    # Testnorm
    
    b=np.array(Prices_test.iloc[:,0])
    A=np.array(Prices_test.iloc[:,1:N+1])
    
    print('Test'+str(Year)+str(Month)+':'+str(norm(A@x-b)))
    
    #Plot
    
    #b=np.append(b,np.array(Prices_test.iloc[:,0]))
    #A=np.append(A,np.array(Prices_test.iloc[:,1:N+1]),axis=0)
    
    b=np.array(Prices.iloc[:,0])
    A=np.array(Prices.iloc[:,1:N+1])
    r=np.array(pd.to_datetime(Prices.index))
    vert1=pd.to_datetime(Prices_train.index[0])
    vert2=pd.to_datetime(Prices_train.index[-1])

    #print(pd.to_datetime(Prices_train.index[-1]),'-',pd.to_datetime(Prices.index[0]))
    #print(vert)
    
    #r=range(b.shape[0])

    plt.figure(i,figsize=(15,5))
    plt.plot(r,A@x, label='rebuild')
    plt.plot(r,b, label='original')
    plt.axvline(vert1)
    plt.axvline(vert2)
    plt.legend()
    plt.title(str(Year)+'_'+str(Month))
    
    return {'TE':{str(Year)+'-'+str(Month):rnorm},'Weights':pd.DataFrame(x,index=UnderlyingPrice.columns,columns=[str(Year)+'-'+str(Month)])}

def MultiPeriodPrice(N,Components,Index,Underlying,NMonth=6):
    i=0
    j=0
    for YearMonth in Components.keys():
        #print(YearMonth)
        Year=int(YearMonth[0:4])
        #print(Year)
        Month=int(YearMonth[4:6])
        #print(Month)
        if j>=NMonth:
            ActDict=SinglePeriodPrice(N,Components,Index,Underlying,Year,Month,NMonth,i)
            if i==0:
                ResDict=ActDict
            else:
                ResDict['TE'].update(ActDict['TE'])
                ResDict['Weights']=pd.merge(ResDict['Weights'],ActDict['Weights'], \
                               how='outer',left_index=True,right_index=True)
            i=i+1
            #print(ResDict)
        j=j+1
        
    
    return ResDict

#### Einfache Optimierung über Return

Funktion zur Bestimmung des Returns über Zeitraum

In [ ]:
def Perform(r,Start=100):
    i=0
    Result=np.empty(1)
    for r_i in r:
        if i==0:
            p_i=Start*(1+r_i)
            Result=np.array(p_i)
        else:
            p_i=p_i*(1+r_i)
            Result=np.append(Result,p_i)
        i=i+1
        #print(str(r_i)+'->'+str(p_i))
    return Result

In [ ]:
def SinglePeriodReturn(N,Components,Index,Underlying,Year,Month,NMonth=6,i=0):
    ActComp=StartComp(N,Components,Year,Month)
    #print(Year,'-',Month,':',ActComp)
    
    IndexReturn=Index[['Date','Daily Return']].set_index('Date')
    UnderlyingReturn=Underlying[Underlying['ISIN'] \
                               .isin(ActComp)][['Date','ISIN','Daily Return']] \
                               .pivot(index='Date',columns='ISIN',values='Daily Return')
    
    Returns=pd.merge(IndexReturn,UnderlyingReturn,left_index=True,right_index=True)
    Returns=Returns.fillna(value=0)
    (YearTrain,MonthTrain)=MonthDelta(Year,Month,-NMonth+1)
    Returns_train=Returns[((pd.to_datetime(Returns.index).year<Year)| \
                  ((pd.to_datetime(Returns.index).month<=Month) & (pd.to_datetime(Returns.index).year==Year))) & \
                       ((pd.to_datetime(Returns.index).year>YearTrain)| \
                  ((pd.to_datetime(Returns.index).month>=MonthTrain) & (pd.to_datetime(Returns.index).year==YearTrain)))]
    (YearTest,MonthTest)=MonthDelta(Year,Month,1)
    #print(YearTest)
    #print(MonthTest)
    Returns_test=Returns[(~Returns.index.isin(Returns_train.index)) & \
                       (pd.to_datetime(Returns.index).month==MonthTest) & \
                       (pd.to_datetime(Returns.index).year==YearTest)]
    
    
    b=np.array(Returns_train.iloc[:,0])
    A=np.array(Returns_train.iloc[:,1:N+1])
    
    vert=b.shape[0]
    
    (x, rnorm)=nnls(A,b)
    
    # Testnorm
    
    b=np.array(Returns_test.iloc[:,0])
    A=np.array(Returns_test.iloc[:,1:N+1])
    
    print('Test'+str(Year)+str(Month)+':'+str(norm(A@x-b)))
    
    #Plot
    
    #b=np.append(b,np.array(Returns_test.iloc[:,0]))
    #A=np.append(A,np.array(Returns_test.iloc[:,1:N+1]),axis=0)
    
    b=np.array(Returns.iloc[:,0])
    A=np.array(Returns.iloc[:,1:N+1])
    r=np.array(pd.to_datetime(Returns.index))
    vert1=pd.to_datetime(Returns_train.index[0])
    vert2=pd.to_datetime(Returns_train.index[-1])

    #print(pd.to_datetime(Returns_train.index[-1]),'-',pd.to_datetime(Returns.index[0]))
    #print(vert)
    
    #r=range(b.shape[0])

    plt.figure(i,figsize=(15,5))
    plt.plot(r,Perform(A@x), label='rebuild')
    plt.plot(r,Perform(b), label='original')
    plt.axvline(vert1)
    plt.axvline(vert2)
    plt.legend()
    plt.title(str(Year)+'_'+str(Month))
    
    return {'TE':{str(Year)+'-'+str(Month):rnorm},'Weights':pd.DataFrame(x,index=UnderlyingReturn.columns,columns=[str(Year)+'-'+str(Month)])}

def MultiPeriodReturn(N,Components,Index,Underlying,NMonth=6):
    i=0
    j=0
    for YearMonth in Components.keys():
        #print(YearMonth)
        Year=int(YearMonth[0:4])
        #print(Year)
        Month=int(YearMonth[4:6])
        #print(Month)
        if j>=NMonth:
            ActDict=SinglePeriodReturn(N,Components,Index,Underlying,Year,Month,NMonth,i)
            if i==0:
                ResDict=ActDict
            else:
                ResDict['TE'].update(ActDict['TE'])
                ResDict['Weights']=pd.merge(ResDict['Weights'],ActDict['Weights'], \
                               how='outer',left_index=True,right_index=True)
            i=i+1
            #print(ResDict)
        j=j+1
        
    
    return ResDict
    

### LAIT - Linear Approximation for Index Tracking Problem

In [ ]:
import math

def d (gamma,p,w):
    return 1/math.log(1+gamma/p)/(p+w)

def LAIT (CompPrice,IndexPrice,w0=None,maxiter=1000,eps=1e-9,p=1e-6,gamma=0.2,lamb=0.5):
    
    T,n=CompPrice.shape
    
    if w0 is None:
        w=np.ones(n) 
        w=w/w.sum() # equal weights for every component
    else:
        if pd.isna(w0):
            raise ValueError('NaNs in start vector')
        else:
            w=w0
    
    # Zählvariable definieren
    k=0
    # conv soll die Verbesserung der Zielfunktion messen
    conv=eps
    TE_last=norm(CompPrice@w-IndexPrice)
    
    dv = lambda x: d(gamma,p,x)
        
    while (conv>=eps) & (k<=maxiter):
        c=dv(w)
        b=np.concatenate((IndexPrice,np.zeros(w.shape)),axis=None)
        A=np.concatenate((CompPrice,lamb*np.diag(c)),axis=0)
        
        w,TE=nnls(A,b)
        
        print('Iteration: ',k,'-->TE:',TE)
        
        conv=abs(TE-TE_last)
        
        TE_last=TE
        k=k+1
    
    return (w,TE)
        

Vorbereitung Anwendung

Components=UnderlyingDaten
Components=Components.set_index(pd.to_datetime(Components.Date)).drop('Date',axis=1)
Components=Components[(Components.index.year==2021) & (Components.index.month ==1)]
#Components=Components.pivot(index='Date',columns='ISIN',values='Index Price')

Index=IndexDaten.set_index(pd.to_datetime(IndexDaten.Date)).drop('Date',axis=1)
Index=Index[(Index.index.year==2021) & (Index.index.month == 1)]
#Index=np.array(Index['Gross Price Index'])

ActComp=np.random.rand(Components.shape[1])
ActComp=ActComp/ActComp.sum()

CompReturn=UnderlyingDaten[['Date','ISIN','Daily Return']]
CompReturn=CompReturn.set_index(pd.to_datetime(CompReturn.Date))
CompReturn=CompReturn[(CompReturn.index.year==2020) & (CompReturn.index.month == 1)]
CompReturn=CompReturn.pivot(index='Date',columns='ISIN',values='Daily Return')
CompReturn=np.array(CompReturn)

IndexReturn=IndexDaten.set_index(pd.to_datetime(IndexDaten.Date))
IndexReturn=IndexReturn[(IndexReturn.index.year==2020) & (IndexReturn.index.month == 1)]
IndexReturn=np.array(IndexReturn['Daily Return'])

ActComp=np.random.rand(CompReturn.shape[1])
ActComp=ActComp/ActComp.sum()

w_LAIT,TE_LAIT=LAIT(CompReturn,IndexReturn)

### SLAIT - Specialized LAIT with upper bound

In [ ]:
def a_HETE(x,M=0.1):
    if abs(x)<=M:
        return 1
    else:
        return M/abs(x)

def b_HETE(x,M=0.1):
    if abs(x)<=M:
        return 0
    else:
        return M(abs(x)-M)

def b_HDR(x,M=0.1):
    if x<=M:
        return 0
    else:
        return M(abs(x)-M)
    
def a_HDR(x,M=0.1):
    if x<0:
        return M/(M-2*x)
    elif x<=M:
        return 1
    else:
        return M/abs(x)

def c(x):
    if x<0:
        return x
    else:
        return 0

def d (gamma,p,w):
    return 1/math.log(1+gamma/p)/(p+w)
    
def A(q,u=1,maxiter=1000):
    print('Start binary search')
    
    # indirect sort of q
    q_sortindex=q.argsort()
    
    i=1
    a=1
    e=len(q)
    K=(a+e)//2
    mue=-(q[q_sortindex[range(K-1)]].sum()+2)/(K)
    A=(mue+q)<0
    w=-(mue*np.ones(len(q))+q)/2
    #print(A.sum(),'==',K,'-- a=',a,' und e=',e)
        
    while (A.sum()!=K) & (i<=maxiter):
        i=i+1
        if A.sum()>K:
            a=K+1
            K=(a+e)//2
        else:
            e=K-1
            K=(a+e)//2
        mue=-(q[q_sortindex[range(K-1)]].sum()+2)/(K)
        A=(mue+q)<0
        #print(A.sum(),'==',K,'-- a=',a,' und e=',e)
        if A.sum()==K or e<a or i>maxiter:
            w=-(mue*np.ones(len(q))+q)/2
            break
        elif A.sum()==0:
             raise ValueError('Only zero weights')
        
    print('Found optimal K without upper bound. ',(w>u).sum(),' components above upper bound')
        
    if (u<1) & (u>0) & ((w>u).sum()>0):
        
        print('Start combined binary search')
        
        if 1/u>len(q):
            raise ValueError('upper bound too low')
            
        c=q+2*u
        
        for k in np.arange(K,len(q)+1):
            #print('K_opt == ',k)
            i=1
            a=1
            e=k
            K1=(a+e)//2
            K2=k-K1            
        
            mue=-(q[q_sortindex[K1:k-1]].sum()-2*K1*u+2)/K2
            B1=(mue+c)<=0
            B2=(0<mue+c) & (mue+c<2*u)
            
            #print(B1.sum(),'==',K1,' -- ',B2.sum(),'==',K2,'-- a=',a,' und e=',e)

            while (a<=e):
                
                i=i+1
                
                if B1.sum()>K1:
                    a=K1+1
                    K1=(a+e)//2
                    K2=k-K1 
                else:
                    e=K1-1
                    K1=(a+e)//2
                    K2=k-K1 
                
                mue=-(q[q_sortindex[K1:k-1]].sum()-2*K1*u+2)/K2
                B1=(mue+c)<=0
                B2=(0<mue+c) & (mue+c<2*u)
                
                #print(B1.sum(),'==',K1,' -- ',B2.sum(),'==',K2,'-- a=',a,' und e=',e)

                if (B1.sum()==K1) & (B2.sum()==K2):
                    w=np.minimum(-(mue*np.ones(len(q))+q)/2,u*np.ones(len(q)))
                    break
                #elif (B1.sum()==0) | (B2.sum()==0):
                    #raise ValueError('Only zero weights')
            
            if (B1.sum()==K1) & (B2.sum()==K2):
                break
        if ((B1.sum()!=K1) or (B2.sum()!=K2)):
            w[q_sortindex[range(math.ceil(1/u))]]=u
            w=w/w.sum()
    
    return np.maximum(w,0)

av_HETE=np.vectorize(a_HETE)
av_HDR=np.vectorize(a_HDR)
bv_HETE=np.vectorize(b_HETE)
bv_HDR=np.vectorize(b_HDR)
cv=np.vectorize(c)
dv=np.vectorize(d) 

In [ ]:
def SLAIT(X,r,lamb=1e-7,u=1,maxiter=1000,measure='ETE',eps=1e-9,w0=None,M=None,gamma=0.2,p=1e-6,thres=1e-3,ret=False):
    
    ################# error control ########################
    IDs=X.keys()
    X=np.array(X)
    r=np.array(r)
    T,n=X.shape # T is days, n number of constituents
    if n==1:
        raise ValueError('Data is univariate!')
    if pd.isna(X).sum()>0|pd.isna(r).sum()>0|pd.isna(lamb)|pd.isna(u):
        raise ValueError('Some arguments contain NaNs')
    if (measure in ['HETE','HDR'])&(pd.isnull(M)|(M<=0)):
        raise ValueError('The huber parameter should be positive')
    if (u>1)|(u<=0):
        raise ValueError('upper bound not in range (0,1]')
    if measure not in ['ETE','HETE','DR','HDR']:
        raise ValueError(measure,' is no valid measure!')
    ########################################################
    
    if w0 is None:
        w=np.ones(n) 
        w=w/w.sum() # equal weights for every component
    else:
        if np.isnan(w0).any():
            raise ValueError('NaNs in start vector')
        else:
            w=w0
    
    k=0
    conv=eps
    
    if measure in ['ETE','DR']: 
        L1=(X.T@X)/T
        lamb_max=np.linalg.eigvalsh(L1).max()
    
    elif measure in ['HETE','HDR']:
        av=av_HETE(r-X@w)
        aDiag=np.eye(T)
        np.fill_diagonal(aDiag,av)
        L2=(X.T@aDiag@X)/T
        lamb_max=np.linalg.eigvalsh(L2).max()
    
    while (conv>=eps) & (k<=maxiter):
        if measure=='ETE':
            q=(2*(L1-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@r)/lamb_max
        elif measure=='DR':
            y=-np.maximum(X@w-r,0)
            q=(2*(L1-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@(y-r))/lamb_max
        elif measure=='HETE':
            av=av_HETE(r-X@w)
            aDiag=np.eye(T)
            np.fill_diagonal(aDiag,av)
            q=(2*(L2-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@aDiag@r)/lamb_max
        else:
            av=av_HDR(r-X@w)
            aDiag=np.eye(T)
            np.fill_diagonal(aDiag,av)
            L3=X.T@aDiag@X/T
            lamb_max=np.linalg.eigvalsh(L3).max()
            q=(2*(L3-lamb_max*np.eye(n))@w+lamb*dv(gamma,p,w)-2/T*X.T@aDiag@(cv(r-X@w-r)))/lamb_max
        w0=w
        w=A(q,u,maxiter)
        #conv=norm(w0-w)
        k=k+1
        
        #Needs python 3.10 
        #match measure:
            #case 'ETE':
            #    print('Iteration: ',k,'--> Zielfunktion: ',ETE(X,r,w),' and w: ',w)
            #case 'DR':
            #    print('Iteration: ',k,'--> Zielfunktion: ',DR(X,r,w),' and w: ',w)
            #case 'HETE':
            #    print('Iteration: ',k,'--> Zielfunktion: ',HETE(X,r,w,M),' and w: ',w)
            #case 'HDR':
            #    print('Iteration: ',k,'--> Zielfunktion: ',HDR(X,r,w,M),' and w: ',w)
        
        if measure=='ETE':
            print('Iteration: ',k,'--> Zielfunktion: ',ETE(X,r,w),' and w: ',w)
            conv=abs(ETE(X,r,w0)-ETE(X,r,w))
        elif measure=='DR':
            print('Iteration: ',k,'--> Zielfunktion: ',DR(X,r,w),' and w: ',w)
            conv=abs(DR(X,r,w0)-DR(X,r,w))
        elif measure=='HETE':
            print('Iteration: ',k,'--> Zielfunktion: ',HETE(X,r,w,M),' and w: ',w)
            conv=abs(HETE(X,r,w0)-HETE(X,r,w))
        else:
            print('Iteration: ',k,'--> Zielfunktion: ',HDR(X,r,w,M),' and w: ',w)
            conv=abs(HDR(X,r,w0)-HDR(X,r,w))
            
    if w.max()>thres:
        w=np.where(w<thres,0,w)
    w=np.nan_to_num(w)
    w=w/w.sum()
    
    W=pd.DataFrame({'ISIN':IDs,'Weight':w})
                
    return W

def ETE(X,r,w):
    T,n=X.shape
    return norm(X@w-r)**2/T

def DR(X,r,w):
    T,n=X.shape
    return norm(np.maximum(r-X@w,0))**2/T

def phi(x,M):
    if x<=abs(M):
        return x**2
    else:
        return M*(2*abs(x)-M)
    
phiv=np.vectorize(phi)

def HETE(X,r,w,M):
    T,n=X.shape
    return np.ones(T).T@phiv(X@w-r,M)/T

def HDR(X,r,w,M):
    T,n=X.shape
    return np.ones(T).T@phiv(np.maximum(r-X@w,0),M)/T

In [ ]:
from platform import python_version

print(python_version())

In [ ]:
def plot_replicated(w,Index,Comp,Ret=True,Start=100):
    plt.figure(i,figsize=(15,5))

    
    return_reb=Comp@w
    return_org=Index
    
    r=range(len(return_reb))
    
    if Ret==False:
        price_reb=return_reb
        price_org=return_org
        return_reb=np.zeros(len(price_reb))
        return_org=np.zeros(len(price_reb))
        for j in r:
            if j==0:
                return_reb[j]=Start
                return_org[j]=Start
            else:
                return_reb[j]=return_reb[j-1]*(1+np.log(price_reb[j]/price_reb[j-1]))
                return_org[j]=return_org[j-1]*(1+np.log(price_org[j]/price_org[j-1]))
    r=np.array(Index.index)
    plt.plot(r,return_reb, label='rebuild')
    plt.plot(r,return_org, label='original')
            
    plt.legend()

W=SLAIT(X=Components.pivot(columns='ISIN',values='Daily Return').fillna(1),r=Index['Daily Return'],measure='ETE',eps=1e-6,M=0.1,maxiter=1000)

CompPrice=UnderlyingDaten
CompPrice=pd.merge(CompPrice,W,on='ISIN')
CompPrice=CompPrice.set_index(pd.to_datetime(CompPrice.Date)).drop('Date',axis=1)
IndexPrice=IndexDaten.set_index(pd.to_datetime(IndexDaten.Date)).drop('Date',axis=1)
w=CompPrice.pivot(columns='ISIN',values='Weight').iloc[1,:].fillna(0)
CompPrice=CompPrice.pivot(columns='ISIN',values='Index Price').fillna(0)
IndexPrice=IndexPrice['Gross Price Index']
plot_replicated(w,IndexPrice,CompPrice,Ret=False)
print('ETE: ',ETE(CompPrice,IndexPrice,w))
print('DR: ',DR(CompPrice,IndexPrice,w))
print('# of Components: ',(w>0).sum(),' of ',CompPrice.shape[1])
print('Maximum Weigth: ',w.max())

IndexReturn=Index['Daily Return']
CompReturn=np.array(Components.pivot(columns='ISIN',values='Daily Return').fillna(1))
w=np.array(W['Weight'])
plot_replicated(w,IndexReturn,CompReturn)
print('ETE: ',ETE(CompReturn,IndexReturn,w))
print('DR: ',DR(CompReturn,IndexReturn,w))
print('# of Components: ',(w>0).sum(),' of ',CompReturn.shape[1])

In [ ]:
class portfolio (object):
    def __init__(self,weight=None,underlying=None,index=None,components=None):
            self.weight=weight
            self.underlying=underlying
            self.index=index
            self.components=components
            
    def plt_replicated(self,Start=100):
        plt.figure(i,figsize=(15,5))
        
        data=pd.merge_asof(self.underlying,self.weight,on='Date',by='ISIN',direction='backward',allow_exact_matches=False) \
        .dropna()
        data['WeightReturn']=data['Weight']*data['Daily Return']
        data=data.groupby('Date').sum()
        data=pd.merge(data,self.index,on='Date')
        
    
        return_reb=np.array(data['WeightReturn'])
        return_org=np.array(data['Daily Return_y'])
    
        r=range(len(return_reb))
    
        for j in range(len(return_reb)):
            if j==0:
                return_reb[j]=Start
                return_org[j]=Start
            else:
                return_reb[j]=return_reb[j-1]*(1+return_reb[j])
                return_org[j]=return_org[j-1]*(1+return_org[j])
        r=data.index
        plt.plot(r,return_reb, label='rebuild')
        plt.plot(r,return_org, label='original')

        plt.legend()
        plt.show()
        
    def number_components(self):
        return self.weight[self.weight['Weight']>0][['Date','ISIN']].groupby('Date').count()
    
    def compare(self,on,date=None):
        if pd.isna(date):
            date=self.weight['Date'].max()
        data=pd.merge(self.components[date.strftime('%Y%m')],self.weight[self.weight['Date']==date],on='ISIN',how='left').fillna(0)
        if self.components[date.strftime('%Y%m')][on].dtype=='float64':
            data[on]=round(data[on],0)
            print('Rebuild ',on,': ',np.average(data[on], weights=data['Weight']))
            print('Index ',on,': ',np.average(data[on], weights=data['Index Weight']))
        data.groupby(on).sum()[['Weight','Index Weight']].plot(kind='bar',figsize=(25,5))
    
    def compare_list(self):
        return list(self.components[list(self.components.keys())[0]].keys())
    
    def ETE(self,ret=True,Start=1):
        data=pd.merge_asof(self.underlying,self.weight,on='Date',by='ISIN',direction='backward',allow_exact_matches=False) \
        .dropna()
        data['WeightReturn']=data['Weight']*data['Daily Return']
        data=data.groupby('Date').sum()
        data=pd.merge(data,self.index,on='Date')
        return_reb=np.array(data['WeightReturn'])
        return_org=np.array(data['Daily Return_y'])
        
        # Vergleich auf Preisbasis
        if ret==False:
            for j in range(len(return_reb)):
                if j==0:
                    return_reb[j]=Start
                    return_org[j]=Start
                else:
                    return_reb[j]=return_reb[j-1]*(1+return_reb[j])
                    return_org[j]=return_org[j-1]*(1+return_org[j])
                
        return norm(return_reb-return_org)**2/len(return_reb)
    
    def DR(self,ret=True,Start=1):
        data=pd.merge_asof(self.underlying,self.weight,on='Date',by='ISIN',direction='backward',allow_exact_matches=False) \
        .dropna()
        data['WeightReturn']=data['Weight']*data['Daily Return']
        data=data.groupby('Date').sum()
        data=pd.merge(data,self.index,on='Date')
        return_reb=np.array(data['WeightReturn'])
        return_org=np.array(data['Daily Return_y'])
        
        # Vergleich auf Preisbasis
        if ret==False:
            for j in range(len(return_reb)):
                if j==0:
                    return_reb[j]=Start
                    return_org[j]=Start
                else:
                    return_reb[j]=return_reb[j-1]*(1+return_reb[j])
                    return_org[j]=return_org[j-1]*(1+return_org[j])
                
        return norm(np.maximum(return_org-return_reb,0))**2/len(return_org)
    
    def trading_cost(self,base=100,c=5e-4):
        data=self.weight.pivot(columns='Date',values='Weight',index='ISIN').fillna(0)
        
        for (i,date) in enumerate(data.columns):
            if i==0:
                cost=0
            cost=cost+c*norm(data.iloc[:,i]-data.iloc[:,i-1])            
                
        return cost*base
    
    def normalize_weight(self,nominal=None,min_piece=None,weight_step=None,inplace=False):
        if not pd.isna(nominal) or not pd.isna(min_piece):
            weight_step=min_piece/nominal
        elif pd.isna(weight_step):
            raise ValueError('No argument given')
            
        data=self.weight.pivot(columns='Date',values='Weight',index='ISIN').fillna(0)
        
        for (i,date) in enumerate(data.columns):
            if i==0:
                data[date]=(round(data[date]/weight_step,0))*weight_step
            else:
                weight_new=(round(data[date]/weight_step))*weight_step
                data.iloc[:,i]=np.where(abs(data.iloc[:,i]-data.iloc[:,i-1])>weight_step,data.iloc[:,i],data.iloc[:,i-1])
                #data.iloc[:,i-1]+((data.iloc[:,i]-data.iloc[:,i-1])//weight_step)*weight_step
        
        data=data.reset_index().melt(value_vars=data.columns,id_vars='ISIN',value_name='Weight')\
            .dropna().reset_index(drop=True)
        if inplace:
            self.weight=data[['ISIN','Weight','Date']]
        else:
            return data[['ISIN','Weight','Date']]
        

In [ ]:
def MultiPeriod_SLAIT(X,r,Comp,IndexISIN='DE0009682716',Startdate=None,Enddate=None,m_rebal_freq=1,MonthRew=1, \
                      lamb=1e-7,u=1,maxiter=1000,measure='ETE' \
                      ,eps=1e-9,w0=None,M=0.1,gamma=0.2,p=1e-6,thres=1e-3,ret=False):

    # Datenvorbereitung
    r=r.sort_values('Date',ignore_index=True)
    X=X.sort_values(['Date','ISIN'],ignore_index=True)

    r=r.set_index(pd.to_datetime(r.Date)).drop('Date',axis=1)
    X=X.set_index(pd.to_datetime(X.Date)).drop('Date',axis=1)
    
    # Standardwert für Zeitraum -> Maximum möglich
    if Startdate==None:
        Startdate=X.index.min()
    if Enddate==None:
        Enddate=X.index.max()
    
    #Filter Indexdaten auf gewünschten Index
    r=r[r['ISIN_TRi']==IndexISIN]
        
    #Initialisierung leeres Portfolio
    port=pd.DataFrame()
    
    # Trennung in Teilzeiträume    
    dates=pd.date_range(Startdate,Enddate,freq='M')
    
    if len(dates)%MonthRew!=0:
        raise ValueError('Number of dates available must be multiple of MothRew')
    
    # Iteration über Zeiträume
    for i in range(MonthRew-1,len(dates)-MonthRew,MonthRew):
        
        #Auswahl der Komponenten für kommenden Zeitraum 
        Components=Comp[dates[i+1].strftime('%Y%m')]
        print('Components: ',Components.shape)

        # Filter auf alle Komponentenzeitreihen des Vorzeitraumes und im neuen noch aktiv sind
        Underlying=X[(X.index>=dates[i-MonthRew+1].replace(day=1)) & (X.index<=dates[i])]
        Underlying=Underlying[Underlying['ISIN'].isin(list(Components['ISIN']))]
        
        #Bestimmung der Startgewichtung: entweder Startvektor oder aber die Gewichtung des Vorzeitraumes
        if port.empty:
            w=w0
        else:
            w=np.array(pd.merge(Underlying,port[port['Date']==port['Date'].max()],on='ISIN',how='left')\
                       .fillna(0).groupby('ISIN').mean()['Weight'])
        
        # Extraktion Return-Zeitreihen für Index und Underlyings
        Underlying=Underlying.pivot(columns='ISIN',values='Daily Return').fillna(1)
        print('Underlying: ',Underlying.shape)
        Index=r[(r.index>=dates[i-MonthRew+1].replace(day=1)) & (r.index<=dates[i])]
        Index=Index['Daily Return']
        print('Index: ',Index.shape)
        
        # Wenn Steuervariable ret = Falsch, dann Umwandlung DailyReturns in fortlaufende Returns
        if ret==False:
            for j in range(len(Index)):
                if j>0:
                    Index[j]=Index[j-1]*(1+Index[j])
                    Underlying.iloc[j,:]=Underlying.iloc[j-1,:]*(1+Underlying.iloc[j,:])
        
        result=SLAIT(X=Underlying,r=Index,u=u,measure=measure,eps=eps,M=M,maxiter=maxiter,lamb=lamb,w0=w,gamma=gamma,p=p,thres=thres)
        result['Date']=dates[i]
        result=result[result['Weight']>0]
        
        
        
        port=port.append(result,ignore_index=True)

    return portfolio(port,X,r,Comp)

In [ ]:
W_1=MultiPeriod_SLAIT(underlyings_df,indices_df,comp,Startdate='2020-01-01',maxiter=10000)

Startdatum='2020-01-01'
Enddatum='2021-03-31'

IndexISIN='DE0009682716'

N=50

IndexDaten=indices_df[(indices_df['ISIN_TRi']==IndexISIN) & 
                      (indices_df['Date']>=Startdatum) & 
                      (indices_df['Date']<=Enddatum)].sort_values('Date',ignore_index=True)
UnderlyingDaten=underlyings_df[(underlyings_df['Date']>=Startdatum) & 
                      (underlyings_df['Date']<=Enddatum)].sort_values(['Date','ISIN'],ignore_index=True)

IndexDaten=IndexDaten.set_index(pd.to_datetime(IndexDaten.Date)).drop('Date',axis=1)
UnderlyingDaten=UnderlyingDaten.set_index(pd.to_datetime(UnderlyingDaten.Date)).drop('Date',axis=1)

In [ ]:
W_2=MultiPeriod_SLAIT(underlyings_df,indices_df,comp,Startdate='2020-01-01',maxiter=10000,u=0.01)

In [ ]:
W_3=MultiPeriod_SLAIT(underlyings_df,indices_df,comp,Startdate='2020-01-01',Enddate='2021-03-31',maxiter=10000,m_rebal_freq=3,MonthRew=3)

In [ ]:
W_4=MultiPeriod_SLAIT(underlyings_df,indices_df,comp,Startdate='2020-01-01',Enddate='2021-03-31',maxiter=10000,m_rebal_freq=3,MonthRew=3,u=0.01)

In [ ]:
W_1.number_components()

In [ ]:
W_2.number_components()

In [ ]:
W_3.number_components()

In [ ]:
W_4.number_components()

In [ ]:
W_4.index['Number of Bonds'].max()

In [ ]:
W_4.index['Number of Bonds'].min()

In [ ]:
W_1.plt_replicated()

In [ ]:
W_2.plt_replicated()

In [ ]:
W_3.plt_replicated()

In [ ]:
W_4.plt_replicated()

In [ ]:
print('Portfolio 1 -> ETE: ',W_1.ETE(),' DR: ',W_1.DR(),' TC: ',W_1.trading_cost())
print('Portfolio 2 -> ETE: ',W_2.ETE(),' DR: ',W_2.DR(),' TC: ',W_2.trading_cost())
print('Portfolio 3 -> ETE: ',W_3.ETE(),' DR: ',W_3.DR(),' TC: ',W_3.trading_cost())
print('Portfolio 4 -> ETE: ',W_4.ETE(),' DR: ',W_4.DR(),' TC: ',W_4.trading_cost())

In [ ]:
W_4.compare('Issuer Country')

In [ ]:
W_4.compare('Annual Modified Duration')

In [ ]:
W_4.compare('Level 2')

In [ ]:
W_4.compare('Coupon')

In [ ]:
W_4.compare('Time to Maturity')

In [ ]:
W_4.compare('Markit iBoxx Rating')

In [ ]:
W_4.compare('Asset Swap Margin')

In [ ]:
W_4.compare('Annual Yield')

In [ ]:
W_4.compare('Country Of Risk')

In [ ]:
W_4.compare_list()

In [ ]:
W_4_Mod=portfolio(W_4.weight,W_4.underlying,W_4.index,W_4.components)

In [ ]:
W_4_Mod.normalize_weight(nominal=300000000,min_piece=100000,inplace=True)

In [ ]:
W_4_Mod.trading_cost()

In [ ]:
print('Portfolio 4 -> ETE: ',W_4_Mod.ETE(),' DR: ',W_4_Mod.DR(),' TC: ',W_4_Mod.trading_cost())

In [ ]:
W_4_Mod.plt_replicated()

In [ ]:
W_4_Mod.number_components()

In [ ]:
W_4_Mod.weight.max()

In [ ]:
W_4_Mod.components['202101'][W_4_Mod.components['202101'].ISIN=='XS2063535970']

In [ ]:
from datetime import datetime 
from dateutil import tz 
W_4.weight.to_csv('results/W_4_Weight_'+datetime.now(tz.gettz('GMT+2')).strftime('%Y%m%d_%H%M%S')+'.csv',sep='\t',decimal=',')
W_1.weight.to_csv('results/W_1_Weight'+datetime.now(tz.gettz('GMT+2')).strftime('%Y%m%d_%H%M%S')+'.csv',sep='\t',decimal=',')
W_2.weight.to_csv('results/W_2_Weight'+datetime.now(tz.gettz('GMT+2')).strftime('%Y%m%d_%H%M%S')+'.csv',sep='\t',decimal=',')
W_3.weight.to_csv('results/W_3_Weight'+datetime.now(tz.gettz('GMT+2')).strftime('%Y%m%d_%H%M%S')+'.csv',sep='\t',decimal=',')

In [ ]:
indices_df[indices_df['BBG_Ticker_TRi']=='I4BL']

In [ ]:
W_4_corp=MultiPeriod_SLAIT(underlyings_df,indices_df,comp,IndexISIN='DE000A0G84N4',Startdate='2020-01-01',Enddate='2021-03-31',maxiter=10000,m_rebal_freq=3,MonthRew=3,u=0.01)

In [ ]:
W_4_ret=MultiPeriod_SLAIT(underlyings_df,indices_df,comp,Startdate='2020-01-01',Enddate='2021-03-31',maxiter=1000,m_rebal_freq=1,MonthRew=1,u=0.01)

In [ ]:
W_4_ret.plt_replicated()

In [ ]:
W_4_ret=portfolio(W_4_ret.weight,W_4_ret.underlying,W_4_ret.index,W_4_ret.components)

In [ ]:
W_4_ret.ETE(ret=False)

In [ ]:
W_4_ret.number_components()